# <span style='color:blue'> RBDO with OpenTURNS </span>

RBDO class is used to solve such reliability-based design optimization problem:

\begin{eqnarray}
\min_{\mathbf{d},\mathbf{p}} & & C(\mathbf{d},\mathbf{p}) \\
\text{s.t.} & & \left\{
                \begin{array}{ll}
                  \mathbb{P} \left(g_i(\mathbf{d},\mathbf{X}(\mathbf{p}),\mathbf{Z})\leq 0 \right) \leq P_{f_i}^T, \;\; i= 1,\ldots,m\\
                  h_j(\mathbf{d},\mathbf{p})\leq 0, \;\; j= 1,\ldots,M\\
                \end{array}
              \right.
\end{eqnarray}
with $\mathbf{d} \in \mathbb{R}^{n_d} $, $\mathbf{p}\in\mathbb{R}^{n_p}$, $\mathbf{X}\sim \mathbf{f}_{\mathbf{X}|\mathbf{d}}(\cdot)$ of dimension $n_X$ and $\mathbf{Z}\sim \mathbf{f}_{\mathbf{Z}}(\cdot)$

- $\mathbf{d}$ is a vector of deterministic variables
- $\mathbf{p}$ is a vector of deterministic variables corresponding to hyperparameters of the random variables $\mathbf{X}$ defined by its joint PDF $\mathbf{f}_{\mathbf{X}|\mathbf{d}}(\cdot)$ (e.g., mean, standard deviation, shape parameters) 
- $\mathbf{X}$ is a random vector with some hyperparameters defining its PDF that can depend on $\mathbf{p}$
- $\mathbf{Z}$ is a random vector with fixed hyperparameters defined by the PDF $\mathbf{f}_{\mathbf{Z}}(\cdot)$
- $P_{f_i}^T$ is a threshold of maximal probability value for the constraint $i$


Three algorithms are currently available in RBDO class to solve such a problem:
- <span style='color:blue'> RIA </span>  (Reliability Index Approach) algorithm
- <span style='color:blue'> PMA </span> (Performance Measure Approach) algorithm with 4 different methods for inverse FORM: 'SLSQP', 'AMV', 'CMV' and 'HMV'
- <span style='color:blue'> SORA </span> (Sequential Optimization and Reliability Assessment) algorithm with 4 different methods for inverse FORM: 'SLSQP', 'AMV', 'CMV' and 'HMV'

Please refer to the following papers for more details on the algorithms:
- Valdebenito, Marcos A., and Gerhart I. Schuëller. "A survey on approaches for reliability-based optimization." Structural and Multidisciplinary Optimization 42.5 (2010): 645-663.
- Aoues, Younes, and Alaa Chateauneuf. "Benchmark study of numerical methods for reliability-based design optimization." Structural and multidisciplinary optimization 41.2 (2010): 277-294.
- Du, Xiaoping, and Wei Chen. "Sequential optimization and reliability assessment method for efficient probabilistic design." J. Mech. Des. 126.2 (2004): 225-233.

## Short column design

A short column with a rectangular cross-section of dimensions $b$ and $h$ is subjected to normal force $F$ and biaxial bending moments $M_1$ and $M_2$. According to the elastic-plastic constitutive law, the limit state function is given by:
\begin{equation}
G(b,h,F,M_1,M_2,f_y) = 1 - \frac{4M_1}{b*h^2f_y} - \frac{4M_2}{b^2*h*f_y} - \frac{F^2}{(b*h*f_y)^2}
\end{equation}
with $f_y$ the yield stress. 
The resulting RBDO problem consists to minimize the cross-section area of the column while ensuring physical integrity of the column.

\begin{eqnarray}
\min_{\mathbf{p}=[\mu_b,\mu_h]} & & A(\mathbf{p}) = \mu_b*\mu_h \\
\text{s.t.} & & \left\{
                \begin{array}{ll}
                  \mathbb{P} \left(G(\mathbf{X}(\mathbf{p}),\mathbf{Z})\leq 0 \right) \leq P_{f}^T \\
                  0.5 \leq \mu_b / \mu_h \leq 2. \\
                  100 \leq \mu_b \leq 1000. \\
                  100 \leq \mu_h \leq 1000. \\
                \end{array}
              \right.
\end{eqnarray}
with $\mathbf{p}=[\mu_b,\mu_h]$, $\mathbf{X} = [b,h]$ and $b\sim \mathcal{N}(\mu_b,0.05\times\mu_b)$, $h\sim \mathcal{N}(\mu_h,0.05\times\mu_h)$. Moreover, $\mathbf{Z} = [F,M_1,M_2,f_y]$ a independant random vector with $F\sim \mathcal{N}(2.5\times10^6,0.2\times2.5\times10^6)$, $M_1\sim \mathcal{N}(250\times10^6,0.3\times250\times10^6)$, $M_2\sim \mathcal{N}(125\times10^6,0.3\times125\times10^6)$ and $f_y\sim \mathcal{N}(40,0.1\times40)$.  $P_{f}^T = \Phi(-\beta=3.) \simeq 0.00135$ with $\Phi(\cdot)$ the CFD of a standard Normal distribution.

See the paper for a complete description of the case:
- Dubourg, Vincent, Bruno Sudret, and Jean-Marc Bourinet. "Reliability-based design optimization using kriging surrogates and subset simulation." Structural and Multidisciplinary Optimization 44.5 (2011): 673-690.
- Aoues, Younes, and Alaa Chateauneuf. "Benchmark study of numerical methods for reliability-based design optimization." Structural and multidisciplinary optimization 41.2 (2010): 277-294.

In [1]:
import openturns as ot
from RBDO_class import *
import time

## Definition of the objective Python function
def obj(d,p):
    p0 = p[0]
    p1 = p[1]

    
    obj_ = p0*p1
    return obj_

active_index_d = []
active_index_p = [0,1]
len_XX = 2
f_obj = Objective(obj,active_index_d,active_index_p,len_XX)


## Definition of the constraint functions
def G1(d,x,z):
    b = x[0]
    h = x[1]

    
    F = z[0]*1e6
    M1 = z[1]*1e6
    M2 = z[2]*1e6
    fy = z[3]
    
    g = 1-(4*M1/(b*h**2*fy))-(4*M2/(b**2*h*fy))-(F**2/(b*h*fy)**2)
    return [g]

## Definition of the constraint functions
def G_determ1(d,p):
    b = p[0]
    h = p[1]
    
    g = 0.5-b/h
    return g

## Definition of the constraint functions
def G_determ2(d,p):
    b = p[0]
    h = p[1]
    
    g = b/h-2.
    return g

active_index_d = []
active_index_p = [0,1]
len_XX = 2
g_determ1 = DIneqCons(G_determ1,active_index_d,active_index_p,len_XX)
g_determ2 = DIneqCons(G_determ2,active_index_d,active_index_p,len_XX)

## Definition of the parametric distributions for X 
def updatedistG1(p):
    p1 = p[0]
    p2 = p[1]

    marg1 = ot.Normal(p1,p1*0.05)
    marg2 = ot.Normal(p2,p2*0.05)
    dist = ot.ComposedDistribution([marg1,marg2])
    return dist

F = ot.Normal(2.5,0.2*2.5)
M1 = ot.Normal(250,0.3*250)
M2 = ot.Normal(125,0.3*125)
fy = ot.Normal(40,40*0.1)

distZ_G1 = ot.ComposedDistribution([F,M1,M2,fy])


## Using RIA algorithm

In [2]:

active_index_d_G1 = [] 
active_index_p_G1 = [0,1] 
PfT_G1 = 1.-ot.Normal().computeCDF(3) 
PIneq_G1 = PIneqCons(G1,active_index_d_G1,active_index_p_G1,updatedistG1,distZ_G1,PfT_G1)

## Definition of the list of constraints
PIneqCons_list = [PIneq_G1]  #list of Probability inequality constraints
DIneqCons_list = [g_determ1,g_determ2] #list of deterministic constraints --> no deterministic ,constraints here
Bounds = ot.Interval([100,100], [1000,1000])  #Definition of the bounds on XX=[d,p] 
len_d = 0 #len of deterministic variable vector d
len_p = 2 #len of hyperparemeters variable vector p

# Definition of RBDO problem using the previously defined objective function, constraint functions and bounds
RBDO_Problem_test = RBDO_Problem(f_obj,PIneqCons_list,DIneqCons_list,Bounds,len_d,len_p)

# Initial optimization points
InitialPoint = [500,500]

# Solver type of RIA
Solver = ot.NLopt('LN_COBYLA')
Solver.setVerbose(True)
Solver.setMaximumEvaluationNumber(1000)
Solver.setMaximumIterationNumber(200)

t0 = time.time()
# Instantiation of the RIA algorithm
RIA_Algorithm_test = RIA_Algorithm(RBDO_Problem_test,Solver,InitialPoint)

# Run of RIA algorithm
result = RIA_Algorithm_test.run()

#Obtained results
print('Optimal design variables p = ',RIA_Algorithm_test.get_optimum())
print('Optimal objective function value = ',RIA_Algorithm_test.get_foptimum())
print('Optimal constraint functions value = ',RIA_Algorithm_test.get_consoptimum())
print('Number of RIA iterations = ', result.getEvaluationNumber())
print('Time CPU = ',time.time()-t0)

Optimal design variables p =  [318.125,636.251]
Optimal objective function value =  [202407]
Optimal constraint functions value =  [0.0013498980316771151, -6.393961582418228e-09, -1.4999999936060384]
Number of RIA iterations =  33
Time CPU =  1.0309865474700928


## Using PMA algorithm

In [3]:
## Using PMA algorithm

active_index_d_G1 = [] 
active_index_p_G1 = [0,1] 
PfT_G1 = 1.-ot.Normal().computeCDF(3) 
InvFORM_solver_G1 = 'SLSQP' ## Choice of Inverse FORM Solver : ['SLSQP','AMV','CMV','HMV']
PIneq_G1 = PIneqCons(G1,active_index_d_G1,active_index_p_G1,updatedistG1,distZ_G1,PfT_G1,solver_invFORM=InvFORM_solver_G1)

## Definition of the list of constraints
PIneqCons_list = [PIneq_G1]  #list of Probability inequality constraints
DIneqCons_list = [g_determ1,g_determ2] #list of deterministic constraints --> no deterministic ,constraints here
Bounds = ot.Interval([100,100], [1000,1000])  #Definition of the bounds on XX=[d,p] 
len_d = 0 #len of deterministic variable vector d
len_p = 2 #len of hyperparemeters variable vector p

# Definition of RBDO problem using the previously defined objective function, constraint functions and bounds
RBDO_Problem_test = RBDO_Problem(f_obj,PIneqCons_list,DIneqCons_list,Bounds,len_d,len_p)

# Initial optimization points
InitialPoint = [500,500]

# Solver type of PMA
Solver = ot.NLopt('LN_COBYLA')
Solver.setVerbose(True)
Solver.setMaximumEvaluationNumber(1000)
Solver.setMaximumIterationNumber(200)

# Instantiation of the PMA algorithm
PMA_Algorithm_test = PMA_Algorithm(RBDO_Problem_test,Solver,InitialPoint)

t0 = time.time()
# Run of PMA algorithm
result = PMA_Algorithm_test.run()

#Obtained results
print('Optimal design variables p = ',PMA_Algorithm_test.get_optimum())
print('Optimal objective function value = ',PMA_Algorithm_test.get_foptimum())
print('Optimal constraint functions value = ',PMA_Algorithm_test.get_consoptimum())
print('Number of PMA iterations = ', result.getEvaluationNumber())
print('Time CPU = ',time.time()-t0)

Optimal design variables p =  [318.125,636.251]
Optimal objective function value =  [202407]
Optimal constraint functions value =  [0.0013498980452265345, -9.223164454397192e-10, -1.4999999990776836]
Number of PMA iterations =  39
Time CPU =  0.6903257369995117


## Using SORA algorithm

In [4]:
## Using SORA algorithm

active_index_d_G1 = [] 
active_index_p_G1 = [0,1] 
PfT_G1 = 1.-ot.Normal().computeCDF(3) 
InvFORM_solver_G1 = 'SLSQP' ## Choice of Inverse FORM Solver : ['SLSQP','AMV','CMV','HMV']
PIneq_G1 = PIneqCons(G1,active_index_d_G1,active_index_p_G1,updatedistG1,distZ_G1,PfT_G1,solver_invFORM=InvFORM_solver_G1)

## Definition of the list of constraints
PIneqCons_list = [PIneq_G1]  #list of Probability inequality constraints
DIneqCons_list = [g_determ1,g_determ2] #list of deterministic constraints --> no deterministic ,constraints here
Bounds = ot.Interval([100,100], [1000,1000])  #Definition of the bounds on XX=[d,p] 
len_d = 0 #len of deterministic variable vector d
len_p = 2 #len of hyperparemeters variable vector p

# Definition of RBDO problem using the previously defined objective function, constraint functions and bounds
RBDO_Problem_test = RBDO_Problem(f_obj,PIneqCons_list,DIneqCons_list,Bounds,len_d,len_p)

# Initial optimization points
InitialPoint = [500,500]

# Solver type of SORA
Solver = ot.NLopt('LN_COBYLA')
Solver.setVerbose(True)
Solver.setMaximumEvaluationNumber(1000)
Solver.setMaximumIterationNumber(200)

t0 = time.time()
# Instantiation of the SORA algorithm
SORA_Algorithm_test = SORA_Algorithm(RBDO_Problem_test,Solver,InitialPoint)

# Run of PMA algorithm
result = SORA_Algorithm_test.run()

#Obtained results
print('Optimal design variables p = ',SORA_Algorithm_test.get_optimum())
print('Optimal objective function value = ',SORA_Algorithm_test.get_foptimum())
print('Optimal constraint functions value = ',SORA_Algorithm_test.get_consoptimum())
print('Number of SORA iterations = ', result.getEvaluationNumber())
print('Time CPU = ',time.time()-t0)

Optimal design variables p =  [318.185,636.128]
Optimal objective function value =  [202406]
Optimal constraint functions value =  [0.001349935720655742, -0.00019013343023210894, -1.499809866569768]
Number of SORA iterations =  26
Time CPU =  0.16782879829406738
